In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

import nltk
from nltk.corpus import stopwords
import string
import re
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.read_excel('train.xlsx')
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,27734,1273,becomes more and more frustrating as the film ...,0
1,27757,1275,"There are cheesy backdrops , ridiculous action...",0
2,27758,1275,"are cheesy backdrops , ridiculous action seque...",0
3,27759,1275,"are cheesy backdrops , ridiculous action seque...",0
4,27760,1275,"cheesy backdrops , ridiculous action sequences...",0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PhraseId    10000 non-null  int64 
 1   SentenceId  10000 non-null  int64 
 2   Phrase      10000 non-null  object
 3   Sentiment   10000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 312.6+ KB


**DATA PREPARATION**

In [ ]:
df['Sentiment'].value_counts()

0    2000
1    2000
2    2000
3    2000
4    2000
Name: Sentiment, dtype: int64

In [ ]:
'''
The sentiment labels are:

0 - negative
1 - somewhat negative
2 - neutral
3 - somewhat positive
4 - positive
'''
df['Phrase'].iloc[0]   # Sentence before preprocessing

'becomes more and more frustrating as the film goes on'

In [ ]:
# FUNCTION FOR REMOVING STOPWORDS, PUNCTUATIONS ETC

def preprocess_text(text):
    # Remove irrelevant characters
    text = text.lower() # Convert to lowercase
    text = re.sub(r'\d+', '', text) # Remove digits
    text = re.sub(r'<.*?>', '', text) # Remove HTML tags
    text = re.sub(r'http\S+', '', text) # Remove URLs
    text = text.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation marks
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = nltk.word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    
    # Join the filtered words to form a sentence
    sentence = " ".join(filtered_text)
    
    return sentence



In [ ]:
# Apply the function to the "text" column of the DataFrame
df["Phrase"] = df["Phrase"].apply(preprocess_text)

In [ ]:
# Initializing tokenizer from BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
df['Phrase'].iloc[0]

'becomes frustrating film goes'

In [ ]:
# Performing tokenization for a sample sentence

token = tokenizer.encode_plus(
    df['Phrase'].iloc[0], 
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)

In [ ]:
#token.input_ids
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [ ]:
X_input_ids.shape

(10000, 256)

In [ ]:
#FUNCTION for populating ids, masks with tokens

def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['Phrase'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [ ]:
# Generating input_ids and attn_masks for training out model

X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)
     

0it [00:00, ?it/s]

In [ ]:
#X_input_ids
labels = np.zeros((len(df), 5))
labels.shape

(10000, 5)

In [ ]:
labels[np.arange(len(df)), df['Sentiment'].values] = 1 # one-hot encoded target tensor

In [ ]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...

dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [ ]:
def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [ ]:
dataset = dataset.map(SentimentDatasetMapFunction) # converting to required format for tensorflow dataset 

In [ ]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [ ]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True)    # Shuffle Dataset

In [ ]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [ ]:
p = 0.8
train_size = int((len(df)//16)*p) # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.

In [ ]:
train_size

500

In [ ]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

MODEL

In [ ]:
from transformers import TFBertModel

In [ ]:
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:
optim = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [ ]:
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs = 10
)

Epoch 1/10
500/500 [==============================] - 427s 853ms/step - loss: 0.7770 - accuracy: 0.7057 - val_loss: 0.5373 - val_accuracy: 0.8140
Epoch 2/10
500/500 [==============================] - 433s 865ms/step - loss: 0.5606 - accuracy: 0.7994 - val_loss: 0.4945 - val_accuracy: 0.8330
Epoch 3/10
500/500 [==============================] - 433s 866ms/step - loss: 0.3657 - accuracy: 0.8796 - val_loss: 0.1954 - val_accuracy: 0.9400
Epoch 4/10
500/500 [==============================] - 433s 866ms/step - loss: 0.2700 - accuracy: 0.9133 - val_loss: 0.1257 - val_accuracy: 0.9620
Epoch 5/10
500/500 [==============================] - 433s 866ms/step - loss: 0.1939 - accuracy: 0.9379 - val_loss: 0.1010 - val_accuracy: 0.9700
Epoch 6/10
500/500 [==============================] - 433s 866ms/step - loss: 0.1602 - accuracy: 0.9476 - val_loss: 0.0772 - val_accuracy: 0.9805
Epoch 7/10
500/500 [==============================] - 440s 880ms/step - loss: 0.1252 - accuracy: 0.9591 - val_loss: 0.0433 -

In [ ]:
sentiment_model.save('sentiment_model')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
sentiment_model.save('/content/gdrive/MyDrive/sentiment_model')

**Prediction**

In [ ]:
# Loading the saved model for performing predictions

sentiment_model = tf.keras.models.load_model('/content/gdrive/MyDrive/sentiment_model')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
#function to prepare testing sample

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

In [ ]:
def make_prediction(model, processed_data):
    probs = model.predict(processed_data)[0]
    return np.argmax(probs)

In [ ]:
input_text = "call for prevention rather than to place blame"
input_text = preprocess_text(input_text)
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data)
classes=['Negative', 'A bit negative', 'Neutral', 'A bit positive', 'Positive']
print(result, classes[result])

1/1 [==============================] - 5s 5s/step
1 A bit negative


**EVALUATING MODEL**


In [ ]:
df = pd.read_excel('test.xlsx')

In [ ]:
# Making prediction for each test sample and saving in a df column corresponding to the actual labels
preds = []

for i in range(len(df)):
  input_text = df['Phrase'][i]
  input_text = preprocess_text(input_text)
  processed_data = prepare_data(input_text, tokenizer)
  result = make_prediction(sentiment_model, processed_data=processed_data)
  preds.append(result)

df['predictions'] = preds

In [ ]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment,predictions
0,102,3,would have a hard time sitting through this one,0,0
1,104,3,have a hard time sitting through this one,0,1
2,158,5,Aggressive self-glorification and a manipulati...,0,1
3,160,5,self-glorification and a manipulative whitewash,0,1
4,202,7,Trouble Every Day is a plodding mess .,0,0


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
#Calculating accuracy, precision, recall, f1

y_true = df['Sentiment']
y_pred = df['predictions']

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print("Accuracy: {:.2f}%".format(accuracy*100))
print("Precision: {:.2f}%".format(precision*100))
print("Recall: {:.2f}%".format(recall*100))
print("F1-Score: {:.2f}%".format(f1*100))

Accuracy: 37.52%
Precision: 41.29%
Recall: 37.52%
F1-Score: 37.70%
